Reading the class label of train data, generating the arrayClassCount array for saving the count of documents for each class, this would give us the prior probability.

In [0]:
trainlabelFile = open('train.label')
labelline = trainlabelFile.readline()

arrayClassCount = [ 0 for i in range(0,21)]

docdict = {}
cnt = 1
while labelline:
    currentlabel = labelline.strip().split()
    classIndex = int(currentlabel[0])
    docdict[cnt] = classIndex
    arrayClassCount[classIndex] += 1
    labelline = trainlabelFile.readline()
    cnt += 1

Reading train data and saving the data in a matrix, where each row represent the count of words across all documents which belongs to one class has seen the words. 

In [0]:
traindataFile = open('train.data')
matrixProbabilityWord = [[0 for j in range(0,61189)] for i in range(0,21) ]
dataline = traindataFile.readline()
while dataline:
    currentdocword = dataline.strip().split()
    docIndex = int(currentdocword[0])
    wordIndex = int(currentdocword[1])
    matrixProbabilityWord[docdict[docIndex]][wordIndex] += 1
    dataline = traindataFile.readline()

Adding 1 to every count to avoid 0 probability

In [0]:
for i in range(1,21):
    for j in range(1,61189):
      matrixProbabilityWord[i][j] += 1

Calculating the likelihoods of individual events 

---

(IIDs)

In [0]:
for i in range(1,21):
    for j in range(1,61189):
      matrixProbabilityWord[i][j] /= arrayClassCount[i]

In [0]:
wordDocumentavailability = [[0 for j in range(0,61189)] for i in range(0,7506)]
testdataFile = open('test.data')

Reading the test data to create the boolean matrix. Each row represents the presence of words.

In [0]:
dataline = testdataFile.readline()
docIndex = 0
while dataline:
    currentdocword = dataline.strip().split()
    docIndex = int(currentdocword[0])
    wordIndex = int(currentdocword[1])
    wordDocumentavailability[docIndex][wordIndex] = 1
    dataline = testdataFile.readline()


In [0]:
total_words = 0
for i in range(0,21):
    total_words += arrayClassCount[i]
print(total_words)

11269


Calculating posterior probabilities, we are taking log of individual likelihoods(IIDs) so that we don't miss information.

In [0]:
import math
testlabelFile = open('test.label')
labelline = testlabelFile.readline()
docId = 1
correctmapping = 0
#print(labelline)


while labelline:
    currentline = labelline.strip().split()
    classlabel = int(currentline[0])
    #print(classlabel)
    maxValue = float('-inf')
    prediction = 0
    for c in range(1,21):
        py = 0
        for word in range(1,61189):
        #print(word)
            if wordDocumentavailability[docId][word] == 1:
                py += math.log(matrixProbabilityWord[c][word])
            else:
                py += math.log(1-matrixProbabilityWord[c][word])
        py += math.log(arrayClassCount[c]/total_words)
        if py > maxValue:
            prediction = c
            maxValue = py

    #print(docId,prediction)
    #print(prediction)
    if  prediction == classlabel:
        correctmapping+= 1
    labelline = testlabelFile.readline()
    docId += 1

print(correctmapping)
                

4678


In [0]:
print(100*correctmapping/7505)

62.331778814123915


The success rate while treating the data as binomial distribution, yields us 62.33 % accuracy.